In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
import csv

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importing dataset

In [4]:
l = []

with open("/content/drive/MyDrive/booksummaries.txt", 'r', encoding="utf8") as file:
    r = csv.reader(file, dialect='excel-tab')
    for row in r:
        l.append(row)

Identifying components of the book from the obtained list and storing the same in a dataframe

In [5]:
Id = []
Name = []
Description = []
Genre = []

In [6]:
for i in l:
    Id.append(i[0])
    Name.append(i[2])
    Genre.append(i[5])
    Description.append(i[6])

df = pd.DataFrame({'id': Id, 'name': Name, 'description': Description,
                       'genre': Genre})

In [7]:
df.head()

,id,name,description,genre
0,620,Animal Farm,"Old Major, the old boar on the Manor Farm, ca...","{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"":..."
1,843,A Clockwork Orange,"Alex, a teenager living in near-future Englan...","{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""N..."
2,986,The Plague,The text of The Plague is divided into five p...,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fi..."
3,1756,An Enquiry Concerning Human Understanding,The argument of the Enquiry proceeds by a ser...,
4,2080,A Fire Upon the Deep,The novel posits that space around the Milky ...,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90..."


In [8]:
df.tail()

,id,name,description,genre
16554,36934824,Under Wildwood,"Prue McKeel, having rescued her brother from ...",
16555,37054020,Transfer of Power,The reader first meets Rapp while he is doing...,"{""/m/01jfsb"": ""Thriller"", ""/m/02xlf"": ""Fiction""}"
16556,37122323,Decoded,The book follows very rough chronological ord...,"{""/m/0xdf"": ""Autobiography""}"
16557,37132319,America Again: Re-becoming The Greatness We Ne...,Colbert addresses topics including Wall Stree...,
16558,37159503,Poor Folk,Makar Devushkin and Varvara Dobroselova are s...,"{""/m/02ql9"": ""Epistolary novel"", ""/m/014dfn"": ..."


In [9]:
df.dtypes

id             object
name           object
description    object
genre          object
dtype: object

In [10]:
df['id'] = df['id'].astype('int')

Dropping rows with empty genre

In [11]:
df.drop(df[df['genre']==''].index, inplace=True)
df[df['genre']=='']

,id,name,description,genre


In [12]:
df.tail()

,id,name,description,genre
16549,36372465,The Third Lynx,The story starts with former government agent...,"{""/m/06n90"": ""Science Fiction""}"
16551,36534061,Remote Control,The series follows the character of Nick Ston...,"{""/m/01jfsb"": ""Thriller"", ""/m/02xlf"": ""Fiction..."
16555,37054020,Transfer of Power,The reader first meets Rapp while he is doing...,"{""/m/01jfsb"": ""Thriller"", ""/m/02xlf"": ""Fiction""}"
16556,37122323,Decoded,The book follows very rough chronological ord...,"{""/m/0xdf"": ""Autobiography""}"
16558,37159503,Poor Folk,Makar Devushkin and Varvara Dobroselova are s...,"{""/m/02ql9"": ""Epistolary novel"", ""/m/014dfn"": ..."


Converting json format of genre to list

In [13]:
import json as jsc

In [14]:
jsc.loads(df['genre'][0]).values()

dict_values(['Roman à clef', 'Satire', "Children's literature", 'Speculative fiction', 'Fiction'])

In [15]:
l=[]
for i in df['genre']:
    l.append(list(jsc.loads(i).values()))

In [16]:
df['u-genre'] = l

In [17]:
setOfgenre = sum(l,[])
setOfgenre = set(setOfgenre)

In [18]:
len(setOfgenre)

227

Preprocessing description text

In [19]:
import re

In [20]:
def updateDescription(s):
    s = re.sub("\'", "", s)
    s = re.sub("[^a-zA-Z]"," ",s)
    s = ' '.join(s.split())
    s = s.lower()
    return s

In [21]:
df['u-description'] = df['description'].apply(lambda x: updateDescription(x))

In [22]:
df.head()

,id,name,description,genre,u-genre,u-description
0,620,Animal Farm,"Old Major, the old boar on the Manor Farm, ca...","{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"":...","[Roman à clef, Satire, Children's literature, ...",old major the old boar on the manor farm calls...
1,843,A Clockwork Orange,"Alex, a teenager living in near-future Englan...","{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""N...","[Science Fiction, Novella, Speculative fiction...",alex a teenager living in near future england ...
2,986,The Plague,The text of The Plague is divided into five p...,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fi...","[Existentialism, Fiction, Absurdist fiction, N...",the text of the plague is divided into five pa...
4,2080,A Fire Upon the Deep,The novel posits that space around the Milky ...,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90...","[Hard science fiction, Science Fiction, Specul...",the novel posits that space around the milky w...
5,2152,All Quiet on the Western Front,"The book tells the story of Paul Bäumer, a Ge...","{""/m/098tmk"": ""War novel"", ""/m/016lj8"": ""Roman...","[War novel, Roman à clef]",the book tells the story of paul b umer a germ...


In [23]:
df=df.drop(['genre', 'description'], axis=1)

In [24]:
df.head()

,id,name,u-genre,u-description
0,620,Animal Farm,"[Roman à clef, Satire, Children's literature, ...",old major the old boar on the manor farm calls...
1,843,A Clockwork Orange,"[Science Fiction, Novella, Speculative fiction...",alex a teenager living in near future england ...
2,986,The Plague,"[Existentialism, Fiction, Absurdist fiction, N...",the text of the plague is divided into five pa...
4,2080,A Fire Upon the Deep,"[Hard science fiction, Science Fiction, Specul...",the novel posits that space around the milky w...
5,2152,All Quiet on the Western Front,"[War novel, Roman à clef]",the book tells the story of paul b umer a germ...


In [25]:
df.rename(columns = {'u-genre':'genre'}, inplace = True)

In [26]:
df.rename(columns = {'u-description':'description'}, inplace = True)

In [27]:
df.head()

,id,name,genre,description
0,620,Animal Farm,"[Roman à clef, Satire, Children's literature, ...",old major the old boar on the manor farm calls...
1,843,A Clockwork Orange,"[Science Fiction, Novella, Speculative fiction...",alex a teenager living in near future england ...
2,986,The Plague,"[Existentialism, Fiction, Absurdist fiction, N...",the text of the plague is divided into five pa...
4,2080,A Fire Upon the Deep,"[Hard science fiction, Science Fiction, Specul...",the novel posits that space around the milky w...
5,2152,All Quiet on the Western Front,"[War novel, Roman à clef]",the book tells the story of paul b umer a germ...


In [28]:
df.dtypes

id              int64
name           object
genre          object
description    object
dtype: object

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12841 entries, 0 to 16558
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           12841 non-null  int64 
 1   name         12841 non-null  object
 2   genre        12841 non-null  object
 3   description  12841 non-null  object
dtypes: int64(1), object(3)
memory usage: 1017.6+ KB


In [30]:
df.shape

(12841, 4)

In [31]:
df.describe()

,id
count,1.284100e+04
mean,9.986379e+06
std,9.032047e+06
min,6.200000e+02
25%,2.626115e+06
50%,6.877746e+06
75%,1.557749e+07
max,3.715950e+07


In [32]:
import nltk
from nltk.corpus import stopwords 
nltk.download('stopwords')
nltk.download('words')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [33]:
from nltk.tokenize import word_tokenize

Removing stopwords from the description

In [34]:
def preprocess_SW(desc):
    setSW = set(stopwords.words('english'))
    tokens = word_tokenize(desc)
    newDesc = [x for x in tokens if not x.lower() in setSW]
    return ' '.join(newDesc)

In [35]:
preprocess_SW("old major the old boar on the manor farm calls the animals on the farm for a meeting where he compares the humans to parasites and teaches the animals a revolutionary song beasts of england when major dies two young pigs snowball and napoleon assume command and turn his dream into a philosophy")

'old major old boar manor farm calls animals farm meeting compares humans parasites teaches animals revolutionary song beasts england major dies two young pigs snowball napoleon assume command turn dream philosophy'

In [36]:
df['description w/o SW'] = df['description'].apply(lambda x: preprocess_SW(x))

In [37]:
df.head()

,id,name,genre,description,description w/o SW
0,620,Animal Farm,"[Roman à clef, Satire, Children's literature, ...",old major the old boar on the manor farm calls...,old major old boar manor farm calls animals fa...
1,843,A Clockwork Orange,"[Science Fiction, Novella, Speculative fiction...",alex a teenager living in near future england ...,alex teenager living near future england leads...
2,986,The Plague,"[Existentialism, Fiction, Absurdist fiction, N...",the text of the plague is divided into five pa...,text plague divided five parts town oran thous...
4,2080,A Fire Upon the Deep,"[Hard science fiction, Science Fiction, Specul...",the novel posits that space around the milky w...,novel posits space around milky way divided co...
5,2152,All Quiet on the Western Front,"[War novel, Roman à clef]",the book tells the story of paul b umer a germ...,book tells story paul b umer german soldier ur...


In [38]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

Encoding multiple labels since each data point can belong to multiple classes

In [39]:
mBin = MultiLabelBinarizer()

In [40]:
mBin.fit(df['genre'])

MultiLabelBinarizer()

In [41]:
label = mBin.transform(df['genre'])

Splitting into training and testing data

In [42]:
trainData, testData, trainLabel, testLabel = train_test_split(df['description w/o SW'],label, random_state=0, train_size = .80)

In [43]:
# plt.figure(figsize=(20,10))
# sns.heatmap(corr, cmap='YlGnBu', annot=True)
# plt.show()

Converting the input text to feature vectors

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [45]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=10000)
trainData_v = tfidf_vectorizer.fit_transform(trainData)
testData_v = tfidf_vectorizer.transform(testData)

In [46]:
from sklearn.multiclass import OneVsRestClassifier
lr = LogisticRegression()
clf = OneVsRestClassifier(lr)
clf.fit(trainData_v, trainLabel)

OneVsRestClassifier(estimator=LogisticRegression())

In [47]:
from sklearn.metrics import f1_score, accuracy_score

In [48]:
y_pred = clf.predict(testData_v)
f1_score(testLabel, y_pred, average="micro"), accuracy_score(testLabel, y_pred)

(0.3657861635220126, 0.08991825613079019)

In [49]:
# pred_prob = clf.predict_proba(testData_v)

In [50]:
# t = 0.3
# predp = (pred_prob >= t).astype(int)
# f1_score(testLabel, predp, average="micro"), accuracy_score(testLabel, predp)

In [51]:
def predict(m):
    m = updateDescription(m)
    m = preprocess_SW(m)
    m_vec = tfidf_vectorizer.transform([m])
    m_pred = clf.predict(m_vec)
    return mBin.inverse_transform(m_pred)

In [1]:
# from random import sample
# for i in range(10):
#     k = sample(list(testData),0)
#     print("Book: ", df['name'][k], 
#           "\nPredicted genre: ", predict(testData[k])) ,
#     print("Actual genre: ",df['genre'][k], "\n")

In [52]:
test_df = pd.DataFrame(testData)
test_df.head()

,description w/o SW
12457,incident saudi arabia touches world war iii un...
1403,british risen power egypt suppress worship old...
6325,year old annabel hope jayquith beautiful famou...
11366,midsomer worthy writers circle group amateur n...
4374,book follows jason worthing also known jazz bo...


In [53]:
predict(test_df['description w/o SW'][1403])

[('Speculative fiction',)]

In [54]:
df['genre'][1403]

['Time travel', 'Speculative fiction', 'Fantasy', 'Steampunk']

In [55]:
df['genre']

0        [Roman à clef, Satire, Children's literature, ...
1        [Science Fiction, Novella, Speculative fiction...
2        [Existentialism, Fiction, Absurdist fiction, N...
4        [Hard science fiction, Science Fiction, Specul...
5                                [War novel, Roman à clef]
                               ...                        
16549                                    [Science Fiction]
16551                        [Thriller, Fiction, Suspense]
16555                                  [Thriller, Fiction]
16556                                      [Autobiography]
16558              [Epistolary novel, Speculative fiction]
Name: genre, Length: 12841, dtype: object

In [56]:
setOfgenre

{'Absurdist fiction',
 'Adventure',
 'Adventure novel',
 'Albino bias',
 'Alien invasion',
 'Alternate history',
 'American Gothic Fiction',
 'Anthology',
 'Anthropology',
 'Anti-nuclear',
 'Anti-war',
 'Apocalyptic and post-apocalyptic fiction',
 'Autobiographical comics',
 'Autobiographical novel',
 'Autobiography',
 'Bangsian fantasy',
 'Bildungsroman',
 'Biographical novel',
 'Biography',
 'Biopunk',
 'Bit Lit',
 'Black comedy',
 "Boys' school stories",
 'Business',
 'Cabal',
 'Campus novel',
 'Catastrophic literature',
 'Chick lit',
 "Children's literature",
 'Chivalric romance',
 'Collage',
 'Colonial United States romance',
 'Comedy',
 'Comedy of manners',
 'Comic book',
 'Comic fantasy',
 'Comic novel',
 'Comic science fiction',
 'Comics',
 'Coming of age',
 'Computer Science',
 'Conspiracy',
 'Conspiracy fiction',
 'Contemporary fantasy',
 'Cookbook',
 'Cozy',
 'Creative nonfiction',
 'Crime Fiction',
 'Cyberpunk',
 'Dark fantasy',
 'Detective fiction',
 'Drama',
 'Dying Earth

Grouping similar genres

In [124]:
genre_ls = []
for i in df['genre']:
  ls = i.copy()
  if 'Adventure novel' in ls:
    ls.remove('Adventure novel')
    ls.append('Adventure')
  if 'Apocalyptic and post-apocalyptic fiction' in ls:
    ls.remove('Apocalyptic and post-apocalyptic fiction')
    ls.append('Speculative fiction')
  if 'Autobiographical comics' in ls:
    ls.remove('Autobiographical comics')
    ls.append('Autobiography')
  if 'Autobiographical novel' in ls:
    ls.remove('Autobiographical novel')
    ls.append('Autobiography')
  if 'Biographical novel' in ls:
    ls.remove('Biographical novel')
    ls.append('Biography')
  if 'Biopunk' in ls:
    ls.remove('Biopunk')
    ls.append('Science fiction')
  if 'Cabal' in ls:
    ls.remove('Cabal')
    ls.append('Horror')
  if 'Comedy of manners' in ls:
    ls.remove('Comedy of manners')
    ls.append('Comedy')
  if 'Conspiracy fiction' in ls:
    ls.remove('Conspiracy fiction')
    ls.append('Conspiracy')
  if 'Social commentary' in ls:
    ls.remove('Social commentary')
    ls.append('Sociology')
  if 'Social criticism' in ls:
    ls.remove('Social criticism')
    ls.append('Sociology')
  if 'Social novel' in ls:
    ls.remove('Social novel')
    ls.append('Sociology')
  if 'Social science fiction' in ls:
    ls.remove('Social science fiction')
    ls.append('Sociology')
  if 'Social sciences' in ls:
    ls.remove('Social sciences')
    ls.append('Sociology')
  if 'Transhumanism' in ls:
    ls.remove('Transhumanism')
    ls.append('Philosophy')
  if 'Treatise' in ls:
    ls.remove('Treatise')
    ls.append('Literature')
  if 'True crime' in ls:
    ls.remove('True crime')
    ls.append('Crime')
  if 'Urban fantasy' in ls:
    ls.remove('Urban fantasy')
    ls.append('Fantasy')
  if 'Urban fiction' in ls:
    ls.remove('Urban fiction')
    ls.append('Dark')
  if 'Utopian' in ls:
    ls.remove('Utopian')
    ls.append('Speculative fiction')
  if 'Vampire' in ls:
    ls.remove('Vampire')
    ls.append('Horror')
  if 'War' in ls:
    ls.remove('War')
    ls.append('Action')
  if 'Whodunit' in ls:
    ls.remove('Whodunit')
    ls.append('Detective')
  if 'Wuxia' in ls:
    ls.remove('Wuxia')
    ls.append('Chinese fiction')
  if 'Youth' in ls:
    ls.remove('Youth')
    ls.append('Young Adult')
  if 'Zombie' in ls:
    ls.remove('Zombie')
    ls.append('Horror')
  if 'Speculative fiction' in ls:
    if 'Fiction' in ls:
      ls.remove('Fiction')
  if 'Science fiction' in ls:
    if 'Fiction' in ls:
      ls.remove('Fiction')
  genre_ls.append(ls)
df['grouped-genre'] = genre_ls


In [125]:
df.head()

,id,name,genre,description,description w/o SW,grouped-genre
0,620,Animal Farm,"[Roman à clef, Satire, Children's literature, ...",old major the old boar on the manor farm calls...,old major old boar manor farm calls animals fa...,"[Roman à clef, Satire, Children's literature, ..."
1,843,A Clockwork Orange,"[Science Fiction, Novella, Speculative fiction...",alex a teenager living in near future england ...,alex teenager living near future england leads...,"[Science Fiction, Novella, Speculative fiction..."
2,986,The Plague,"[Existentialism, Fiction, Absurdist fiction, N...",the text of the plague is divided into five pa...,text plague divided five parts town oran thous...,"[Existentialism, Fiction, Absurdist fiction, N..."
4,2080,A Fire Upon the Deep,"[Hard science fiction, Science Fiction, Specul...",the novel posits that space around the milky w...,novel posits space around milky way divided co...,"[Hard science fiction, Science Fiction, Specul..."
5,2152,All Quiet on the Western Front,"[War novel, Roman à clef]",the book tells the story of paul b umer a germ...,book tells story paul b umer german soldier ur...,"[War novel, Roman à clef]"


In [126]:
mBin = MultiLabelBinarizer()

In [127]:
mBin.fit(df['grouped-genre'])

MultiLabelBinarizer()

In [128]:
label = mBin.transform(df['grouped-genre'])

In [129]:
trainData, testData, trainLabel, testLabel = train_test_split(df['description w/o SW'],label, random_state=0, train_size = .80)

In [130]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=10000)
trainData_v = tfidf_vectorizer.fit_transform(trainData)
testData_v = tfidf_vectorizer.transform(testData)

In [131]:
from sklearn.multiclass import OneVsRestClassifier
lr = LogisticRegression()
clf = OneVsRestClassifier(lr)
clf.fit(trainData_v, trainLabel)

OneVsRestClassifier(estimator=LogisticRegression())

In [132]:
y_pred = clf.predict(testData_v)
f1_score(testLabel, y_pred, average="micro"), accuracy_score(testLabel, y_pred)

(0.3559816590245936, 0.10548851693265862)

In [133]:
pred_prob = clf.predict_proba(testData_v)

In [134]:
t = 0.3
predp = (pred_prob >= t).astype(int)
f1_score(testLabel, predp, average="micro"), accuracy_score(testLabel, predp)

(0.522585992540406, 0.16543402101985208)

In [116]:
def predict(m):
    m = updateDescription(m)
    m = preprocess_SW(m)
    m_vec = tfidf_vectorizer.transform([m])
    m_pred = clf.predict(m_vec)
    return mBin.inverse_transform(m_pred)

In [117]:
test_df = pd.DataFrame(testData)
test_df.head(20)

,description w/o SW
12457,incident saudi arabia touches world war iii un...
1403,british risen power egypt suppress worship old...
6325,year old annabel hope jayquith beautiful famou...
11366,midsomer worthy writers circle group amateur n...
4374,book follows jason worthing also known jazz bo...
5335,moist von lipwig bored job postmaster general ...
7494,agent pendergast visits medicine creek kansas ...
427,story set primarily around melbourne australia...
14619,october brother cadfael monk shrewsbury abbey ...
10193,winter day dalton new england town near boston...


In [118]:
predict(test_df['description w/o SW'][15526])

[('Science Fiction',)]

In [119]:
df['grouped-genre'][15526]

['Science Fiction']

In [95]:
predict(test_df['description w/o SW'][2069])

[('Fantasy', 'Science Fiction', 'Speculative fiction')]

In [96]:
df['grouped-genre'][2069]

['Science Fiction', 'Speculative fiction', 'Fantasy']

In [122]:
predict(test_df['description w/o SW'][5129])

[('Fantasy', 'Horror', 'Science Fiction', 'Speculative fiction')]

In [123]:
df['grouped-genre'][5129]

['Speculative fiction', 'Horror', "Children's literature"]